# CNN to classify Cifar-10 dataset (Images)



So far, we saw how to build a Dense Neural Network (DNN) that classified images of digits (MNIST) or even fashion images (Fashion-MNIST). Here we will instead, recognize the 10 classes of CIFAR ('airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship' and 'truck'). There are some key differences between these two image datasets that we need to take into account. 

First, while MNIST were 28x28 monochrome images (1 color channel), CIFAR is 32x32 color images (3 color channels).

Second, MNIST images are simple, containing just the object centered in the image, with no background. Conversely, CIFAR ones are not centered and can have the object with a background, such as airplanes that might have a cloudy sky behind them! Those differences are the main reason to use a CNN instead of a DNN. 

## Import Libraries

In [48]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten

## Import and Inspect Dataset

Cifar-10 repository: https://www.cs.toronto.edu/~kriz/cifar.html





In [67]:
cifar10 = datasets.cifar10
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

In [ ]:
print(train_images.shape, train_labels.shape)
print(test_images.shape, test_labels.shape)

- The image data shape is: `(#images, img_heigth, img_width, #channels)`, where channels are in RGB format (red, green, blue). 
- The labels shape is `(#images, label)`, where label goes from 0 to 9.


In [ ]:
train_images[0]

In [ ]:
plt.imshow(train_images[1]);

In [ ]:
train_labels[1][0]

    The CIFAR labels happen to be arrays, which is why you need the extra index

In [102]:
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [ ]:
class_names[9] # The List's index is the label

In [ ]:
idx = train_labels[1][0]
class_names[idx]

In [ ]:
print("\t", class_names[train_labels[1][0]])
plt.imshow(train_images[1])
plt.axis('off');

In [28]:
 def plot_train_img(img, size=2): 
    label = train_labels[img][0]
    plt.figure(figsize=(size,size))
    print("Label {} - {}".format(label, class_names[label]))
    plt.imshow(train_images[img])
    plt.axis('off')
    plt.show()

In [ ]:
plot_train_img(1)

In [ ]:
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i])
    plt.xlabel(class_names[train_labels[i][0]])
plt.show()

Note that images are in color, not centered and with different backgrounds

## Preprocessing dataset

In [ ]:
test_images.max()

In [71]:
# Normalize pixel values to be between 0 and 1
train_images = train_images / 255.0
test_images = test_images / 255.0

In [ ]:
test_images.max()

In [ ]:
plt.hist(train_labels[:5_000]);

In [ ]:
val_images = train_images[:5_000]
val_labels = train_labels[:5_000]
print(val_images.shape, val_labels.shape)

In [ ]:
train_images = train_images[5_000:]
train_labels = train_labels[5_000:]
print(train_images.shape, train_labels.shape)

In [ ]:
plt.hist(train_labels, alpha=0.5)
plt.hist(val_labels, alpha=0.5)
plt.hist(test_labels, alpha=0.5);

## Create Model Arquitecture and Compile

On [Convolution layers](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2D), 
- strides is an integer or tuple/list of 2 integers, specifying the strides of the convolution along the height and width. Default (1,1).
- padding: one of "valid" or "same" (case-insensitive). Default = 'valid'.
  - "valid" means no padding.  
  - "same" results in padding with zeros evenly
to the left/right or up/down of the input such that output has the same


In [ ]:
model = Sequential()


model.add(Conv2D(
    filters=32, 
    kernel_size=(3,3), 
    activation='relu', 
    input_shape=(32, 32, 3))
)
model.add(MaxPool2D(2, 2))

model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPool2D())

model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(10, activation='softmax'))

model.summary()

In [53]:
LOSS = 'sparse_categorical_crossentropy'
OPTIMIZER = 'adam'

# Compile the model
model.compile(optimizer=OPTIMIZER,
              loss=LOSS,
              metrics=['accuracy'])

## Training

In [78]:
NUM_EPOCHS = 20 #You can change this value if you like to experiment with it to get better accuracy

In [ ]:
# Fit the model
history = model.fit(train_images, 
                    train_labels, 
                    epochs=NUM_EPOCHS, 
                    validation_data=(val_images, val_labels)
)

In [ ]:
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.xlim([0,NUM_EPOCHS])
plt.ylim([0.4,1.0])
plt.show()

## Evaluate Model

In [ ]:
model.evaluate(test_images, test_labels)

**Accuracy**
- Train: 85% - 90%; 
- Validation: 68%-70% 
- Test: 66%-68%

In [ ]:
predictions = np.argmax(model.predict(test_images), axis=-1)
predictions.shape

In [83]:
from sklearn.metrics import classification_report,confusion_matrix

In [ ]:
print(classification_report(test_labels, predictions, target_names=class_names))

In [ ]:
confusion_matrix(test_labels,predictions)

In [ ]:
class_names

In [ ]:
import seaborn as sns
plt.figure(figsize=(15,8))
sns.heatmap(confusion_matrix(test_labels,predictions), cmap='Blues', annot=True, fmt='g');

## Testing Model (Predicting)

In [ ]:
plt.imshow(test_images[15]);

In [ ]:
test_labels[15][0]

In [ ]:
class_names[8]

In [ ]:
test_images[15].shape

The input Tensor shape should be: (num_images, width, height, color_channels)

In [ ]:
my_image = test_images[15]
my_image = my_image.reshape(1,32,32,3)
my_image.shape

In [ ]:
img_pred = np.argmax(model.predict(my_image))
class_names[img_pred]

In [112]:
def img_pred(img, size=4):
    label = test_labels[img][0]
    my_image = test_images[img]
    plt.figure(figsize=(size,size))
    plt.imshow(my_image)
    my_image = my_image.reshape(1,32,32,3)
    img_pred = np.argmax(model.predict(my_image))
    pred_label = class_names[img_pred]
    print(" Label {} <=> Pred: {}".format(class_names[label], pred_label))
    plt.grid(False)
    plt.axis('off')
    plt.show()

In [ ]:
img_pred(0)

In [ ]:
img_pred(1)

In [ ]:
img_pred(2)

In [ ]:
img_pred(3)

In [ ]:
img_pred(4)

In [ ]:
img_pred(5)

## Saving the model

In [ ]:
!pwd # Linux command, shows where we are in CoLab's folders

In [126]:
model.save('cifar_10_model.h5')

Use [Netron](https://netron.app) to visualize the model, hyperparameters, tensor shapes, etc. Netron is a viewer for neural network, deep learning and machine learning models (See [GitHub](https://github.com/lutzroeder/netron) for instructions about instalation in your desktop). 